# Pandas Training

## Import libraries, datasets

In [12]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

dataset is housing data in boston

In [96]:
boston = pd.read_csv("../data/boston_housing.csv")

## Boston

In [97]:
boston

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [98]:
boston.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

1. Add the first and second floor square footage columns to get total ('1stFlrSF', '2ndFlrSF'), then estimate average area by building type (BldgType') and average quality in a 1-5 rating ('OverallQual', 'OverallCond')

In [167]:
ft2m = 0.092

Without merge

In [202]:
(boston
 # add new columns
 .assign(TotalArea = lambda df: (df['1stFlrSF'] + df['2ndFlrSF']) * ft2m)
 .assign(StarRating = lambda df: ((df['OverallQual'] + df['OverallCond'])/4))

 # group by building type
 .groupby('BldgType', as_index=False).agg({'TotalArea':'mean', 'StarRating':'mean'})
 .assign(TotalArea = area.TotalArea.astype(int))
 .round(2)
 
 # select only ratings above 2.5
 #.loc[lambda df: df.StarRating > 2.5,["BldgType","TotalArea"]]
)



,BldgType,TotalArea,StarRating
0,1Fam,141,2.94
1,2fmCon,141,2.67
2,Duplex,144,2.47
3,Twnhs,117,2.88
4,TwnhsE,120,2.94


With merge

In [204]:
(boston
 .assign(TotalArea = lambda df: (df['1stFlrSF'] + df['2ndFlrSF']) * ft2m)
 .groupby('BldgType', as_index=False).agg({'TotalArea':'mean'})
 .merge(boston
    .assign(StarRating = lambda df: ((df['OverallQual'] + df['OverallCond'])/4))
     .groupby('BldgType', as_index=False).agg({ 'StarRating':'mean'})
    ,on='BldgType'
    ,how="left"
       )
)
 

,BldgType,TotalArea,StarRating
0,1Fam,141.064430,2.943238
1,2fmCon,141.430710,2.669355
2,Duplex,144.192308,2.471154
3,Twnhs,117.259349,2.883721
4,TwnhsE,120.604737,2.940789


2. change the quality to a star rating with the following
- 1 star = quality 1, 2 & 3
- 2 star = 4 & 5
- 3 star = 6
- 4 star = 7, 8 & 9
- 5 star = 10

Using replace/map

In [257]:
( boston
 .assign(Stars = lambda df: df.OverallQual.map({1:1, 2:1, 3:1, 4:2,5:2,6:3,7:4,8:4,9:4,10:5}))
)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,Stars
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,2,2008,WD,Normal,208500,4
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,5,2007,WD,Normal,181500,3
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,9,2008,WD,Normal,223500,4
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000,4
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,12,2008,WD,Normal,250000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,8,2007,WD,Normal,175000,3
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000,3
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500,4
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,4,2010,WD,Normal,142125,2


Using dict, zip, map

In [267]:
ograting = list(range(1,11))
ograting
stars = [1,1,1,2,2,3,4,4,4,5]

rating_dict = dict(zip(ograting,stars))
rating_dict

{1: 1, 2: 1, 3: 1, 4: 2, 5: 2, 6: 3, 7: 4, 8: 4, 9: 4, 10: 5}

In [269]:
( boston
 .assign(Stars = lambda df: df.OverallQual.map(dict(zip(ograting,stars))))
)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,Stars
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,2,2008,WD,Normal,208500,4
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,5,2007,WD,Normal,181500,3
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,9,2008,WD,Normal,223500,4
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000,4
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,NaN,NaN,NaN,0,12,2008,WD,Normal,250000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,8,2007,WD,Normal,175000,3
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000,3
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500,4
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,NaN,NaN,NaN,0,4,2010,WD,Normal,142125,2


3. select three columns, rename them to 'mod_(col name)'

In [311]:
(boston
 .rename(dict(zip(boston.columns,["mod_"+x for x in boston.columns])),axis=1)
)

,mod_Id,mod_MSSubClass,mod_MSZoning,mod_LotFrontage,mod_LotArea,mod_Street,mod_Alley,mod_LotShape,mod_LandContour,mod_Utilities,...,mod_PoolArea,mod_PoolQC,mod_Fence,mod_MiscFeature,mod_MiscVal,mod_MoSold,mod_YrSold,mod_SaleType,mod_SaleCondition,mod_SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


# Shopping List


In [227]:
greengrocer = pd.DataFrame(columns = ['stuk','prijs'],
                        data = [
                            ['appelen', 3.0],
                            ['bananas', 1.5],
                            ['aardbei', 4.5],
                            ['sinasappel', 2.0],
                            ['blauwe bessen', 4.6]
                        ])
greengrocer

,stuk,prijs
0,appelen,3.0
1,bananas,1.5
2,aardbei,4.5
3,sinasappel,2.0
4,blauwe bessen,4.6


In [235]:
oogst = pd.DataFrame(columns = ['stuk','seizoen'],
                        data = [

                            ['appelen', 'winter'],
                            ['bananas', 'lente'],
                            ['aardbei',  'zomer'],
                            ['sinasappel', 'zomer'],
                            ['peren', 'herfst'],
                            ['sinasappel', 'lente'],
                            
                        ])
oogst

,stuk,seizoen
0,appelen,winter
1,bananas,lente
2,aardbei,zomer
3,sinasappel,zomer
4,peren,herfst
5,sinasappel,lente


## .merge

In [238]:
groceries = greengrocer.merge(oogst, how ='right', on ='stuk' )
groceries

# note duplication of sinasappel

,stuk,prijs,seizoen
0,appelen,3.0,winter
1,bananas,1.5,lente
2,aardbei,4.5,zomer
3,sinasappel,2.0,zomer
4,sinasappel,2.0,lente
5,peren,NaN,herfst


In [240]:
groceries = greengrocer.merge(oogst, how ='outer', on ='stuk' )
groceries

,stuk,prijs,seizoen
0,appelen,3.0,winter
1,bananas,1.5,lente
2,aardbei,4.5,zomer
3,sinasappel,2.0,zomer
4,sinasappel,2.0,lente
5,blauwe bessen,4.6,NaN
6,peren,NaN,herfst


## .loc

In [244]:
groceries.loc[lambda df: df.seizoen == 'zomer',:]

,stuk,prijs,seizoen
2,aardbei,4.5,zomer
3,sinasappel,2.0,zomer
